## Notebook to download the AF2 equivalent region of PDB structure
For a given PDB structure, do:
- For each chain:
    - Query PDB API for uniprot_id for this chain
    - Query AF2 database for the AF2 model corresponding to this chain, then download it
    - Load AF2 model and PDB structures into python
    - Get the sequence of PDB structure as well as AF2 model, compute sequence alignment, the residues in AF2 model that do not align with PDB structure are deleted
    - Align the AF2 model to the PDB structure to reconstruct the complex


### Define helper functions for the AF2 structure download and alignment workflow:

In [3]:
# Function to get the uniprot_id for a given PDB chain
def get_uniprot_id_from_pdb_chain(pdb_id, chain_id):
    """
    Queries the PDBe API to get the UniProt accession for a given PDB ID and chain.

    Args:
        pdb_id (str): The PDB ID (e.g., '7MON').
        chain_id (str): The chain identifier (e.g., 'A').

    Returns:
        str or None: The UniProt ID if found, else None.
    """
    import requests

    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id.lower()}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for PDB {pdb_id}")
        return None
    data = response.json()
    mappings = data.get(pdb_id.lower(), {}).get("UniProt", {})
    for uniprot_id, mapping in mappings.items():
        for segment in mapping.get("mappings", []):
            if segment.get("chain_id") == chain_id:
                return uniprot_id
    print(f"UniProt ID not found for chain {chain_id} in PDB {pdb_id}")
    return None

#get_uniprot_id_from_pdb_chain("7MON", "A")

In [4]:
# Function to download an AF2 model using the uniprot_id and load it into python (using a temp file)
def fetch_af2_model(uniprot_id):
    """
    Downloads an AlphaFold2 (AF2) model by querying the AlphaFold DB API using the UniProt ID,
    saves the corresponding PDB file to a temporary location, and loads it into Python using Biopython.

    Args:
        uniprot_id (str): The UniProt ID (e.g., 'Q8NB16').

    Returns:
        tuple: (pdb_url, structure)
            pdb_url (str or None): The URL to the downloaded PDB file or None if not found.
            structure: The Biopython Structure object parsed from the downloaded AF2 PDB file,
                       or None if download/parsing failed.
    """
    import requests
    import tempfile
    from Bio.PDB import PDBParser

    # Query the AlphaFold API to get pdbUrl
    api_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
    try:
        api_response = requests.get(api_url)
        if api_response.status_code != 200:
            print(f"Failed to query AlphaFold API for {uniprot_id} (HTTP {api_response.status_code})")
            return None, None
        api_json = api_response.json()
        if not api_json or 'pdbUrl' not in api_json[0]:
            print(f"No pdbUrl found in AlphaFold API response for {uniprot_id}")
            return None, None
        pdb_url = api_json[0]['pdbUrl']
    except Exception as e:
        print(f"Error fetching/parsing AlphaFold API response: {e}")
        return None, None

    # Download the PDB file to a temporary location
    try:
        pdb_response = requests.get(pdb_url)
        if pdb_response.status_code != 200:
            print(f"Failed to download PDB file for {uniprot_id} at {pdb_url} (HTTP {pdb_response.status_code})")
            return pdb_url, None
        with tempfile.NamedTemporaryFile(suffix=".pdb", delete=True) as tmp_fp:
            tmp_fp.write(pdb_response.content)
            tmp_fp.flush()
            parser = PDBParser(QUIET=True)
            structure = parser.get_structure(uniprot_id, tmp_fp.name)
            return pdb_url, structure
    except Exception as e:
        print(f"Failed to download or parse PDB file for {uniprot_id}: {e}")
        return pdb_url, None

# af2_pdb_url, af2_struct = fetch_af2_model("Q8NB16")

In [6]:
# Function to get protein sequence from a biopython structure
def get_protein_sequence(structure):
    """
    Extracts the protein sequence from a Biopython structure.

    Args:
        structure: Biopython Structure object

    Returns:
        dict: A mapping from chain ID to its one-letter protein sequence.
    """
    from Bio.PDB.Polypeptide import is_aa
    from Bio.Data.IUPACData import protein_letters_3to1

    # Build map from 3-letter (upper) codes to 1-letter codes
    _3to1 = {k.upper(): v for k, v in protein_letters_3to1.items()}

    sequences = {}
    for model in structure:
        for chain in model:
            seq = ""
            for residue in chain:
                if is_aa(residue, standard=True):
                    resname = residue.get_resname().upper()
                    try:
                        seq += _3to1[resname]
                    except KeyError:
                        # Skip residues that can't be translated (e.g., unknowns)
                        continue
            if seq:
                sequences[chain.id] = seq
        break  # Only first model (usually sufficient for X-ray/NMR)
    return sequences

# sequences = get_protein_sequence(af2_struct)
# sequences

In [7]:
# Function to get a single chain structure from a biopython structure
def get_single_chain_structure(structure, chain_id):
    # from Bio.PDB import Structure, Model, Chain

    # # Create a new structure containing only the selected chain
    # model = structure[0]
    # selected_chain = model[chain]

    # Build a new Structure object with a single model and the filtered chain
    from Bio.PDB.StructureBuilder import StructureBuilder
    builder = StructureBuilder()
    builder.init_structure("filtered")
    builder.init_model(0)
    # Add only the selected chain to the new structure
    builder.structure[0].add(structure[0][chain_id])
    return builder.get_structure()

In [8]:
# Function to align two protein sequences using EMBOSS needle
# EMBOSS needle is needed rather than Bio.Align.PairwiseAligner because it recognizes gaps in the alignment correctly
import subprocess
import tempfile
from pathlib import Path
from textwrap import wrap
import re

# Function to align two protein sequences using EMBOSS needle
def run_needle_alignment(
    test_seq: str,
    ref_seq: str,
    test_id: str = "test_seq",
    ref_id: str = "ref_seq",
    gapopen: float = 10.0,
    gapextend: float = 0.5,
    needle_exe: str = "needle",
    aformat: str = "pair",
) -> str:
    """
    Run EMBOSS needle to align two protein sequences.

    Parameters
    ----------
    test_seq, ref_seq : str
        Protein sequences (plain one-letter codes, no FASTA header).
    test_id, ref_id : str
        Sequence identifiers for FASTA headers.
    gapopen : float
        Gap opening penalty for needle.
    gapextend : float
        Gap extension penalty for needle.
    needle_exe : str
        Name or path to the 'needle' executable.
    aformat : str
        Output alignment format (e.g. 'pair', 'markx3', 'markx10', 'fasta').

    Returns
    -------
    str
        The alignment output from needle (text).
    """

    def to_fasta(seq: str, header: str) -> str:
        # Wrap sequence at 60 columns for nice FASTA
        wrapped = "\n".join(wrap(seq.replace("\n", "").strip(), 60))
        return f">{header}\n{wrapped}\n"

    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)

        a_file = tmpdir / "a_seq.fasta"
        b_file = tmpdir / "b_seq.fasta"

        # Write FASTA files
        a_file.write_text(to_fasta(test_seq, test_id))
        b_file.write_text(to_fasta(ref_seq, ref_id))

        # Ask needle to write to stdout
        cmd = [
            needle_exe,
            "-asequence", str(a_file),
            "-bsequence", str(b_file),
            "-gapopen", str(gapopen),
            "-gapextend", str(gapextend),
            "-aformat", aformat,
            "-stdout",           # send alignment to stdout
            "-auto",             # no interactive prompts
        ]

        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=False,  # we'll handle errors ourselves
        )

        if result.returncode != 0:
            raise RuntimeError(
                f"needle failed with code {result.returncode}.\n"
                f"STDOUT:\n{result.stdout}\n\nSTDERR:\n{result.stderr}"
            )

        return result.stdout


# Function to get the range of the aligned sequence in the reference sequence
def get_match_seq_range(
    alignment_text: str,
    test_id: str = "test_seq",
    ref_id: str = "ref_seq",
):
    """
    From a needle 'pair' alignment, return the residue indices in ref_seq
    that align to the first and last *aligned* residues in test_seq.

    "Aligned residue" here means a column where BOTH test and ref have
    a non-gap amino acid (not '-').

    Returns
    -------
    (int | None, int | None)
        (first_ref_idx, last_ref_idx), **0-based** indices in ref_seq.
        If no aligned residue pair exists, returns (None, None).
    """

    aligned_test = []
    aligned_ref = []

    # Collect the aligned strings for test and ref
    for line in alignment_text.splitlines():
        line = line.rstrip()
        if not line or line.startswith("#"):
            continue

        parts = line.split()
        label = parts[0] if parts else ""

        if label == test_id or label == ref_id:
            # Grab the contiguous alignment chunk: letters + '-'
            m = re.search(r'([A-Z\-]+)', line)
            if not m:
                continue
            seq_chunk = m.group(1)

            if label == test_id:
                aligned_test.append(seq_chunk)
            elif label == ref_id:
                aligned_ref.append(seq_chunk)

    aligned_test = "".join(aligned_test)
    aligned_ref = "".join(aligned_ref)

    if len(aligned_test) != len(aligned_ref):
        raise ValueError("Aligned test and ref sequences have different lengths.")

    # i_ref will be 0-based index in the ungapped ref sequence
    i_test = -1  # 0-based index in ungapped test seq, start at -1 before first residue
    i_ref = -1   # 0-based index in ungapped ref seq

    first_ref_idx = None
    last_ref_idx = None

    for a_t, a_r in zip(aligned_test, aligned_ref):
        if a_t != "-":
            i_test += 1
        if a_r != "-":
            i_ref += 1

        # A true aligned residue pair: both non-gaps
        if a_t != "-" and a_r != "-":
            if first_ref_idx is None:
                first_ref_idx = i_ref  # already 0-based
            last_ref_idx = i_ref       # update as we go

    return first_ref_idx, last_ref_idx


# alignment_text = run_needle_alignment(chain_pdb_seq[chain], chain_af2_seq['A'], test_id="pdb", ref_id="af2")

# match_range = get_match_seq_range(alignment_text, test_id="pdb", ref_id="af2")
# print(match_range)

# match_seq = chain_af2_seq['A'][match_range[0]:match_range[1]+1]
# print(match_seq)


In [9]:
# Function to filter a biopython structure to only include a residue range
def filter_residue_range(structure, start, end):
    """
    Filter the given Bio.PDB structure so that only residues with resseq in [start, end] and hetflag == ' ' remain.
    This function modifies the structure in-place and returns it for convenience.
    """

    for model in structure:
        for chain in model:
            all_res = list(chain)
            for residue in all_res:
                hetflag, resseq, icode = residue.id
                # Only keep ' ' (blank) hetflag (standard residues) and in range
                if hetflag != " ":
                    chain.detach_child(residue.id)
                elif not (start <= resseq <= end):
                    chain.detach_child(residue.id)

    return structure

In [10]:
# Function to align the filtered AF2 model to the PDB structure
def align_af2_to_pdb(chain_pdb_struct, chain_af2_struct_filtered):
    """
    Align the filtered AF2 model to the PDB structure using Biopython's Superimposer,
    by performing sequence alignment and only using aligned residues for superimposition.

    Parameters:
        chain_pdb_struct: Bio.PDB.Structure.Structure - structure containing the PDB chain
        chain_af2_struct_filtered: Bio.PDB.Structure.Structure - structure containing the filtered AF2 chain

    Returns:
        chain_af2_struct_aligned: Bio.PDB.Structure.Structure - AF2 structure aligned to the PDB structure
        rmsd: float - RMSD value after alignment (in Angstroms)
    """

    from Bio.PDB import Superimposer
    from Bio import pairwise2
    from Bio.SeqUtils import seq1
    import copy

    # Deepcopy the AF2 structure so the original is not rotated in-place
    af2_struct_aligned = copy.deepcopy(chain_af2_struct_filtered)

    # Get chains for alignment (use first chain in each structure)
    pdb_chain = next(chain for chain in chain_pdb_struct[0])
    af2_chain = next(chain for chain in af2_struct_aligned[0])

    # Convert residue objects to lists and sequences for the relevant chains
    pdb_residues = [res for res in pdb_chain if res.id[0] == ' ' and 'CA' in res]
    af2_residues = [res for res in af2_chain if res.id[0] == ' ' and 'CA' in res]

    # Derive sequences from these residues
    def get_seq_from_residues(residues):
        return "".join(seq1(res.get_resname()) for res in residues)

    pdb_seq = get_seq_from_residues(pdb_residues)
    af2_seq = get_seq_from_residues(af2_residues)

    # Run global sequence alignment
    aln = pairwise2.align.globalxx(pdb_seq, af2_seq, one_alignment_only=True)[0]
    aln_pdb = aln.seqA
    aln_af2 = aln.seqB

    # Select only aligned (non-gap) C-alpha atoms
    pdb_ca_atoms = []
    af2_ca_atoms = []
    pdb_idx = 0
    af2_idx = 0

    for a, b in zip(aln_pdb, aln_af2):
        if a != '-' and b != '-':
            pdb_ca_atoms.append(pdb_residues[pdb_idx]['CA'])
            af2_ca_atoms.append(af2_residues[af2_idx]['CA'])
        if a != '-':
            pdb_idx += 1
        if b != '-':
            af2_idx += 1

    # Ensure we have matching pairs
    if len(pdb_ca_atoms) != len(af2_ca_atoms):
        raise ValueError(f"After alignment, found {len(pdb_ca_atoms)} matched C-alpha pairs but expected lengths to agree.")

    # Superimpose AF2 (moving) onto PDB (fixed)
    sup = Superimposer()
    sup.set_atoms(pdb_ca_atoms, af2_ca_atoms)
    sup.apply(af2_chain.get_atoms())

    rmsd = sup.rms

    return af2_struct_aligned, rmsd

# af2_struct, rmsd = align_af2_to_pdb(chain_pdb_struct, chain_af2_struct_filtered)

In [35]:
#-------- Function to align the AF2 model to the PDB structure --------#

# Function to process AF2 alignment for a single chain
def process_af2_alignment_single_chain(structure, pdb_id, chain):
    """
    Download the equivalent AF2 model of a chain, align it to the PDB structure,
    and return relevant alignment objects/results.

    Args:
        structure: Biopython structure object for the PDB.
        pdb_id: PDB ID string (e.g., '7MON').
        chain: Chain ID (str, e.g., 'A').

    Returns:
        (chain_id, aligned_af2_chain_struct, rmsd)
    """
    # Get the PDB and AF2 structure for a given chain
    chain_pdb_struct = get_single_chain_structure(structure, chain)
    chain_uniprot_id = get_uniprot_id_from_pdb_chain(pdb_id, chain)
    chain_af2_pdb_url, chain_af2_struct = fetch_af2_model(chain_uniprot_id)

    # Get the PDB and AF2 sequence for a given chain
    chain_pdb_seq = get_protein_sequence(chain_pdb_struct)
    chain_af2_seq = get_protein_sequence(chain_af2_struct)

    # Get the residue range in the AF2 model that matches the PDB sequence
    alignment_text = run_needle_alignment(chain_pdb_seq[chain], chain_af2_seq['A'], test_id="pdb", ref_id="af2")
    match_range = get_match_seq_range(alignment_text, test_id="pdb", ref_id="af2")

    # Filter the AF2 model to only include the matched residues
    chain_af2_struct_filtered = filter_residue_range(chain_af2_struct, match_range[0], match_range[1])

    # Align the filtered AF2 model to the PDB structure
    chain_af2_struct_aligned, af2_to_pdb_rmsd = align_af2_to_pdb(chain_pdb_struct, chain_af2_struct_filtered)

    return chain, chain_af2_struct_aligned, af2_to_pdb_rmsd, chain_af2_pdb_url

from Bio.PDB import Structure, Model

# Function to process AF2 alignment for all chains, and return the full aligned structure and rmsd results
def process_af2_alignment(structure, pdb_id, chain_ids):
    """
    For all chain IDs in chain_ids, perform AF2 alignment, and collect results.

    Args:
        structure: Biopython structure object for the PDB.
        pdb_id: PDB ID string (e.g., '7MON').
        chain_ids: List of chain IDs (e.g., ['A','B']).

    Returns:
        full_structure: Structure object containing all aligned AF2 chains together as separate chains.
        results: Dictionary mapping chain ID to a dictionary with RMSD value and AF2 PDB URL.
    """
    # Prepare a new structure object to hold all aligned chains
    full_structure = Structure.Structure('aligned_af2')
    model = Model.Model(0)
    full_structure.add(model)

    results = {}

    for chain in chain_ids:
        chain_id, aligned_chain_struct, rmsd, chain_af2_pdb_url = process_af2_alignment_single_chain(structure, pdb_id, chain)
        # We expect aligned_chain_struct to have a single chain (id, etc.)
        # Add this chain to our model
        for ch in aligned_chain_struct.get_chains():
            # Remove existing chains in the model with same id to avoid duplicates
            if model[chain_id] if chain_id in model else None:
                model.detach_child(chain_id)
            ch.id = chain_id  # Make sure the chain id matches requested chain id
            model.add(ch)
        results[chain_id] = {
            "rmsd": rmsd,
            "af2_pdb_url": chain_af2_pdb_url
        }

    return full_structure, results

# Example usage:
# af2_aligned_struct, rmsd_per_chain = process_af2_alignment(structure, pdb_id, interactor_1_chain_ids)


## Workflow starts here:
_____________

In [30]:
# Example pdb complex:
# PDB accession: 7MON
# Interactor 1: chain A
# Interactor 2: chain B
pdb_complex_id = '7MON_A_B'

pdb_out_dir = '/scratch/ymeng/masif-neosurf-af2/data/pdb'
rmsd_out_dir = '/scratch/ymeng/masif-neosurf-af2/data/rmsd'

!mkdir -p $pdb_out_dir
!mkdir -p $rmsd_out_dir

In [36]:
import os
from Bio.PDB import PDBList, PDBParser
import json
from Bio.PDB import PDBIO

# split the pdb complex id into its constituent parts
pdb_id = pdb_complex_id.split('_')[0]
interactor_1 = pdb_complex_id.split('_')[1]
interactor_2 = pdb_complex_id.split('_')[2]

# Convert interactor_1 and interactor_2 into a list of chain ids, each letter in the string is a chain id
interactor_1_chain_ids = list(interactor_1)
interactor_2_chain_ids = list(interactor_2) 

# Download the pdb file using the pdb_id
pdbl = PDBList()
pdb_file_path = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')

# Load the structure using Biopython's PDBParser
parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, pdb_file_path)

# Align the AF2 model to the PDB structure
interactor_1_af2_aligned_struct, interactor_1_af2_results = process_af2_alignment(structure, pdb_id, interactor_1_chain_ids)
interactor_2_af2_aligned_struct, interactor_2_af2_results = process_af2_alignment(structure, pdb_id, interactor_2_chain_ids)

# Combine interactor_1_af2_rmsd and interactor_2_af2_rmsd into a tiered JSON object
rmsd = {
    'pdb_id': pdb_id,
    'interactor_1': interactor_1_af2_results,
    'interactor_2': interactor_2_af2_results
}

# Write interactor_1_af2_aligned_struct and interactor_2_af2_aligned_struct to pdb files using PDBIO correctly
interactor_1_af2_aligned_struct_pdb_path = os.path.join(pdb_out_dir, f'{pdb_id}_{interactor_1}.pdb')
interactor_2_af2_aligned_struct_pdb_path = os.path.join(pdb_out_dir, f'{pdb_id}_{interactor_2}.pdb')

# The correct way is to provide a filename, not an open file handle, to PDBIO.save
io1 = PDBIO()
io1.set_structure(interactor_1_af2_aligned_struct)
io1.save(interactor_1_af2_aligned_struct_pdb_path)

io2 = PDBIO()
io2.set_structure(interactor_2_af2_aligned_struct)
io2.save(interactor_2_af2_aligned_struct_pdb_path)

# Write rmsd to a json file
rmsd_path = os.path.join(rmsd_out_dir, f'{pdb_id}_rmsd.json')
with open(rmsd_path, 'w') as f:
    json.dump(rmsd, f)


Structure exists: './pdb7mon.ent' 


In [37]:
rmsd

{'pdb_id': '7MON',
 'interactor_1': {'A': {'rmsd': np.float64(4.08641252532503),
   'af2_pdb_url': 'https://alphafold.ebi.ac.uk/files/AF-Q8NB16-F1-model_v6.pdb'}},
 'interactor_2': {'B': {'rmsd': np.float64(3.114038339154029),
   'af2_pdb_url': 'https://alphafold.ebi.ac.uk/files/AF-Q9Y572-F1-model_v6.pdb'}}}

In [23]:
af2_aligned_struct, rmsd_per_chain = process_af2_alignment(structure, pdb_id, ['A', 'B'])

In [25]:
rmsd_per_chain

{'A': np.float64(4.08641252532503), 'B': np.float64(3.114038339154029)}

### Visualization
_________________

In [12]:
# ----------- Helper function to visualize structure with py3Dmol ----------- #
from Bio.PDB import PDBParser
import py3Dmol
from Bio.PDB import PDBIO
from io import StringIO

def add_struct_to_py3dmol(structure, view=None):
    """
    Convert a Bio.PDB Structure to PDB string and add it to a given py3Dmol view.
    If no view is passed, creates and returns a new py3Dmol view.
    """
    io = PDBIO()
    io.set_structure(structure)
    pdb_buf = StringIO()
    io.save(pdb_buf)
    pdb_str = pdb_buf.getvalue()
    
    if view is None:
        view = py3Dmol.view(width=400, height=400)
    view.addModel(pdb_str, 'pdb')
    return view

In [29]:
# Create a view for visualization
view = py3Dmol.view(width=500, height=500)

# Add matched protein structure
add_struct_to_py3dmol(structure, view)
view.setStyle({'model': 0}, {'cartoon': {'color': 'cyan'}})

# Add aligned AF2 model
add_struct_to_py3dmol(interactor_1_af2_aligned_struct, view)
view.setStyle({'model': 1}, {'cartoon': {'color': 'green'}})

# Add aligned AF2 model
add_struct_to_py3dmol(interactor_2_af2_aligned_struct, view)
view.setStyle({'model': 2}, {'cartoon': {'color': 'pink'}})

# Set view parameters
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.